In [ ]:
import yfinance as yf
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 12]


In [ ]:
stonks = ['SPY']
df = yf.download(stonks)
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,24.608616,1003200
1993-02-01,43.96875,44.25000,43.96875,44.25000,24.783640,480500
1993-02-02,44.21875,44.37500,44.12500,44.34375,24.836166,201300
1993-02-03,44.40625,44.84375,44.37500,44.81250,25.098690,529400
1993-02-04,44.96875,45.09375,44.46875,45.00000,25.203709,531500


In [ ]:
def create_bins(df,num_states):
    bins=np.linspace(df['Close'].min(),df['Close'].max(),num_states+1)
    df['State']=np.digitize(df['Close'],bins)-1
    return df, bins

In [ ]:
def transition_matrix(df,num_states):
    matrix=np.zeros((num_states,num_states))
    for i in range(len(df)-1):
        current_state=df['State'].iloc[i]
        next_state=df['State'].iloc[i+1]
        matrix[current_state-1][next_state-1]+=1
    row_sums=matrix.sum(axis=1,keepdims=True)
    row_sums[row_sums==0]=1  # Avoid division by zero
    matrix/=row_sums
    return matrix

In [ ]:
def simulate_markov_chain(matrix,states,start_state,steps):
    current_state=start_state
    simulation=[current_state]
    for _ in range(steps):
        current_state=np.random.choice(states,p=matrix[current_state])
        simulation.append(current_state)
    return simulation


In [ ]:
def state_to_price(simulation,bins):
    prices=[(bins[state]+bins[state+1])/2 for state in simulation]
    return prices

In [ ]:
split=int(0.8*len(df))
train_df=df.iloc[:split]
test_df=df.iloc[split:]

In [ ]:
train_df,bins=create_bins(train_df,500)
test_df['State']=np.digitize(test_df['Close'],bins)-1


In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
matrix=transition_matrix(train_df,500)
states=list(range(500))



In [ ]:
start_state=test_df['State'].iloc[0]
simulation=simulate_markov_chain(matrix,states,start_state,len(test_df)-1)
simulated_prices=state_to_price(simulation,bins)

In [ ]:
plt.plot(test_df['Close'].values,label='Actual Price',color='blue')
plt.plot(simulated_prices,label='Simulated Price',color='pink')
plt.legend()
plt.show()